In [1]:
import h2o

In [2]:
import pandas as pd
from h2o.frame import H2OFrame

Подготовим данные

In [3]:
# Загрузка датасета
data = pd.read_csv('Walmart.csv')

data['Date'] = pd.to_datetime(data['Date'], dayfirst=1)


# Определяем времена года
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'autumn'

# Применяем функцию к колонке с датой
data['season'] = data['Date'].apply(get_season)

# Создаем бинарные колонки для каждого времени года
data = pd.get_dummies(data, columns=['season'], prefix='', prefix_sep='')

data.drop('Date', axis=1)

data = data.drop('Date', axis=1) # Удаляем даты



In [5]:
h2o.init() # Инициализация h2o

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.351-b10, mixed mode)
  Starting server from D:\soft\Anaconda\envs\DZ3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\LP\AppData\Local\Temp\tmpnfgldvqq
  JVM stdout: C:\Users\LP\AppData\Local\Temp\tmpnfgldvqq\h2o_LP_started_from_python.out
  JVM stderr: C:\Users\LP\AppData\Local\Temp\tmpnfgldvqq\h2o_LP_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Minsk
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,3 months and 2 days
H2O_cluster_name:,H2O_from_python_LP_6gmop7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.501 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [6]:
# Преобразование в H2OFrame
hf = h2o.H2OFrame(data)

# Разделение на обучающую и тестовую выборки (80% / 20%)
train, test = hf.split_frame(ratios=[0.8], seed=42)

# Указываем целевую переменную
target = "Weekly_Sales"
features = [col for col in hf.columns if col != target]

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [33]:
from h2o.estimators import H2OGradientBoostingEstimator

# Создаём модель GBM
gbm = H2OGradientBoostingEstimator(
    ntrees=100,         # Количество деревьев
    max_depth=5,        # Глубина деревьев
    learn_rate=0.1,     # Скорость обучения
    sample_rate=0.8,    # Сэмплирование выборки
    col_sample_rate=0.8 # Сэмплирование признаков
)

# Обучение модели
gbm.train(x=features, y=target, training_frame=train)

# Оценка модели
performance = gbm.model_performance(test_data=test)
print(performance)
r2 = gbm.r2()
print(f'R2-score: {r2}')

gbm Model Build progress: |

██████████████████████████████████████████████████████| (done) 100%
ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 17400852476.775955
RMSE: 131912.29084803263
MAE: 78779.16774965209
RMSLE: 0.11654768753613556
Mean Residual Deviance: 17400852476.775955
R2-score: 0.9561650444449035


In [34]:
# save the model
model_path = h2o.save_model(model=gbm, force=True)

In [36]:
# load the model
saved_model = h2o.load_model(model_path)

In [37]:
saved_model

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_model_python_1738669631958_7


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    100                100                         39583                  5            5            5             18            32            26.83

ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 14012177050.866282
RMSE: 118373.04190932275
MAE: 69966.21013244712
RMSLE: 0.10227644818160214
Mean Residual Deviance: 14012177050.866282

Scoring History: 
     timestamp            duration    number_of_trees    training_rmse       training_mae        training_deviance
---  -------------------  ----------  -----------------  ------------------  ------------------  -------------------
     2025-02-04 16:07:35  0.009 sec   0.0                565382.7095504855   468490.2833378898   319657608258.6487
     2025-02-04 16:07:35  0.028 sec   1.0                532356.5394468108   441190.9603022511   283403485091.7838
     2025-02-04 16:07:35  0.032 sec   2.0                505977.3925139255   419307.0836769843   256013121735.191
     2025-02-04 16:07:35  0.035 sec   3.0                472240.316445885    390913.3564549777   223010916476.90958
     2025-02-04 16:07:35  0.038 sec   4.0                444634.00111474126  367258.19225451193  197699394947.3037
     2025-02-04 16:07:35  0.041 sec   5.0                424586.4252357382   350418.64361779543  180273632494.4631
     2025-02-04 16:07:35  0.045 sec   6.0                402800.58113626955  330699.70384969923  162248308163.7165
     2025-02-04 16:07:35  0.049 sec   7.0                381800.73397722904  310668.62895400735  145771800465.5508
     2025-02-04 16:07:35  0.052 sec   8.0                364536.0978689426   294356.9070808267   132886566649.51529
     2025-02-04 16:07:35  0.056 sec   9.0                346834.2188353361   278929.6421987192   120293975355.1178
---  ---                  ---         ---                ---                 ---                 ---
     2025-02-04 16:07:35  0.251 sec   91.0               123843.08467872447  73363.2798885358    15337109622.741718
     2025-02-04 16:07:35  0.254 sec   92.0               123424.66490970046  72877.14607692121   15233647908.071842
     2025-02-04 16:07:35  0.257 sec   93.0               122258.88749982048  72267.380024379     14947235572.693762
     2025-02-04 16:07:35  0.259 sec   94.0               121740.64729166117  71792.96652435475   14820785202.99265
     2025-02-04 16:07:35  0.262 sec   95.0               120951.4588951598   71370.22768290316   14629255408.867529
     2025-02-04 16:07:35  0.264 sec   96.0               120352.37927228214  71068.74120051427   14484695196.499249
     2025-02-04 16:07:35  0.267 sec   97.0               119760.91003708549  70798.03878202019   14342675572.910883
     2025-02-04 16:07:35  0.270 sec   98.0               119069.20782552537  70421.48623374733   14177476252.198153
     2025-02-04 16:07:35  0.273 sec   99.0               118796.44473671731  70183.68707245779   14112595282.083931
     2025-02-04 16:07:35  0.276 sec   100.0              118373.04190932275  69966.21013244712   14012177050.866282
[101 rows x 7 columns]


Variable Importances: 
variable      relative_importance    scaled_importance    percentage
------------  ---------------------  -------------------  ------------
Store         4.91806e+15            1                    0.708816
CPI           1.12842e+15            0.229444             0.162633
Unemployment  6.50834e+14            0.132336             0.0938016
Temperature   1.03743e+14            0.0210942            0.0149519
Fuel_Price    8.01222e+13            0.0162914            0.0115476
winte

Получены метрики:
MSE: 17400852476.775955
R2-score: 0.9561650444449035